Importing Libraries

In [4]:
import cv2
import mediapipe as mp
import math

Helper function to draw lines with shadows

In [5]:
#  Helper Function: Draw Thin Text With Shadow
def draw_text_with_shadow(image, text, pos, font_scale, color, thickness):
    x, y = pos
    # Draw white shadow
    cv2.putText(image, text, (x+1, y+1), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255,255,255), thickness+1, cv2.LINE_AA)
    # Draw main text
    cv2.putText(image, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)

Initiallizing mediapipe and Drawing Parameters

In [6]:
# Initialize MediaPipe drawing and holistic models
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh  # For face mesh connections

Addition of body and facial parameters

In [7]:
important_face_landmarks = {
    "Forehead": 10, "Eyes": 33, "Nose": 1, "Lips": 61, "Chin": 199,
    "Left Ear": 127, "Right Ear": 356
}
important_body_landmarks = {
    "Left Shoulder": 11, "Right Shoulder": 12,
    "Left Elbow": 13, "Right Elbow": 14,
    "Left Wrist": 15, "Right Wrist": 16,
    "Left Hip": 23, "Right Hip": 24,
    "Left Knee": 25, "Right Knee": 26,
    "Left Ankle": 27, "Right Ankle": 28
}
finger_landmarks = {
    0: "Palm", 4: "Thumb Tip", 8: "Index Tip",
    12: "Middle Tip", 16: "Ring Tip", 20: "Pinky Tip"
}
offsets = {
    "Left": (-25, -12),
    "Right": (23, -10),
    "default": (10, -14)
}

Styling Part

In [8]:
# Drawing styles for thin, clean visuals
line_thickness = 1
circle_radius = 2
font_scale = 0.35
text_thickness = 1

#color theme
color_themes = [
    {'dot': (0, 0, 255), 'line': (0, 0, 0)},        # Red dots, black lines
    {'dot': (0, 255, 0), 'line': (160, 32, 240)},   # Green dots, purple lines
    {'dot': (255, 127, 36), 'line': (30, 30, 30)}   # Organe dots, dark gray lines
]
theme_idx = 0

Get Realtime Webcam (full screen)

In [9]:
cap = cv2.VideoCapture(0)

window_name = 'Raw Webcam Feed'
# Create a named window and set it to full screen
cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()  # Reading feed from webcam
    if not ret:
        break
    cv2.imshow(window_name, frame)  # Display the "frame" image in full screen
    if cv2.waitKey(10) & 0xFF == ord('q'):  # Press 'q' to exit the webcam
        break

cap.release()
cv2.destroyAllWindows()  # Exiting the webcam

Make Detections from the feed
 -> Detection of Facial Points
 -> Detection of points on the hand
 -> Detection of body points

In [10]:
cap = cv2.VideoCapture(0)
window_name = 'Raw Webcam Feed'
cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

paused = False
img_count = 0

with mp_holistic.Holistic(
    min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        if not paused:
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            h, w, _ = image.shape

            color_theme = color_themes[theme_idx]

            # --- FACE points ---
            if results.face_landmarks:
                for name, idx in important_face_landmarks.items():
                    try:
                        lm = results.face_landmarks.landmark[idx]
                        x, y = int(lm.x * w), int(lm.y * h)
                        cv2.circle(image, (x, y), circle_radius, color_theme['dot'], -1, lineType=cv2.LINE_AA)
                        dx, dy = offsets["default"]
                        draw_text_with_shadow(image, name, (x + dx, y + dy), font_scale, (0,0,0), text_thickness)
                    except IndexError:
                        continue

            # --- BODY joints ---
            if results.pose_landmarks:
                for name, idx in important_body_landmarks.items():
                    try:
                        lm = results.pose_landmarks.landmark[idx]
                        x, y = int(lm.x * w), int(lm.y * h)
                        cv2.circle(image, (x, y), circle_radius + 1, color_theme['dot'], -1, lineType=cv2.LINE_AA)
                        if "Left" in name: dx, dy = offsets["Left"]
                        elif "Right" in name: dx, dy = offsets["Right"]
                        else: dx, dy = offsets["default"]
                        draw_text_with_shadow(image, name, (x + dx, y + dy), font_scale, (0,0,0), text_thickness)
                    except IndexError:
                        continue
                # Draw pose connection lines
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness, circle_radius=circle_radius),
                    mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness)
                )

            # --- HANDS: L/R hand dots, black lines, offset labels ---
            for hand_side, hand_landmarks in [("Right", results.right_hand_landmarks), ("Left", results.left_hand_landmarks)]:
                if hand_landmarks:
                    for idx, part in finger_landmarks.items():
                        try:
                            lm = hand_landmarks.landmark[idx]
                            x, y = int(lm.x * w), int(lm.y * h)
                            cv2.circle(image, (x, y), circle_radius + 1, color_theme['dot'], -1, lineType=cv2.LINE_AA)
                            dx, dy = offsets[hand_side]
                            draw_text_with_shadow(image, f"{hand_side} {part}", (x + dx, y + dy), font_scale, (0,0,0), text_thickness)
                        except IndexError:
                            continue
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness, circle_radius=circle_radius),
                        mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness)
                    )

            # --- EXTRAS ---
            # A. Simple Gesture: Right hand open
            if results.right_hand_landmarks:
                tip_ids = [8, 12, 16, 20]
                wrist_y = results.right_hand_landmarks.landmark[0].y
                tips_up = 0
                for idx in tip_ids:
                    if results.right_hand_landmarks.landmark[idx].y < wrist_y:
                        tips_up += 1
                if tips_up >= 4:
                    draw_text_with_shadow(image, "Right hand OPEN!", (40, 60), 0.7, (0,80,0), 2)

            # B. Live Elbow Angle (Right)
            def calc_angle(a, b, c):
                ang = math.degrees(
                    math.atan2(c[1]-b[1], c[0]-b[0]) -
                    math.atan2(a[1]-b[1], a[0]-b[0])
                )
                ang = abs(ang)
                if ang > 180: ang = 360 - ang
                return int(ang)
            if results.pose_landmarks:
                try:
                    rs = results.pose_landmarks.landmark[12]
                    re = results.pose_landmarks.landmark[14]
                    rw = results.pose_landmarks.landmark[16]
                    pt_rs = (int(rs.x * w), int(rs.y * h))
                    pt_re = (int(re.x * w), int(re.y * h))
                    pt_rw = (int(rw.x * w), int(rw.y * h))
                    elbow_angle = calc_angle(pt_rs, pt_re, pt_rw)
                    draw_text_with_shadow(image, f"Right Arm Angle: {elbow_angle} deg",
                                         (pt_re[0]+30, pt_re[1]+30), 0.5, (180,55,55), 1)
                except Exception:
                    pass

            # C. AR Glasses (rect between both eyes)
            if results.face_landmarks:
                try:
                    eye_left = results.face_landmarks.landmark[33]
                    eye_right = results.face_landmarks.landmark[263]
                    pt_left = (int(eye_left.x * w), int(eye_left.y * h))
                    pt_right = (int(eye_right.x * w), int(eye_right.y * h))
                    cv2.rectangle(image, (pt_left[0]-35, pt_left[1]-10), (pt_right[0]+35, pt_right[1]+25), (75,75,180), 2)
                except Exception:
                    pass

            # D. Pose Confidence
            if results.pose_landmarks:
                scores = [lm.visibility for lm in results.pose_landmarks.landmark]
                avg_score = sum(scores) / len(scores)
                draw_text_with_shadow(image, f"Pose Confidence: {avg_score:.2f}", (15, 40), 0.5, (0, 0, 200), 2)

            # E. Virtual Ruler: Distance between wrists
            if results.pose_landmarks:
                lw = results.pose_landmarks.landmark[15]
                rw = results.pose_landmarks.landmark[16]
                x1, y1 = int(lw.x * w), int(lw.y * h)
                x2, y2 = int(rw.x * w), int(rw.y * h)
                cv2.line(image, (x1, y1), (x2, y2), (200, 0, 0), 1, cv2.LINE_AA)
                dist_px = int(((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5)
                draw_text_with_shadow(image, f"Dist: {dist_px}px", (min(x1, x2)+12, min(y1, y2)-12), 0.45, (0,0,0), 2)

            # F. On-screen help overlay
            draw_text_with_shadow(image,
              "q=quit | space=pause | s=screenshot | c=color theme | Pose conf, R-arm angle, right hand open/AR glasses/ruler shown",
              (8, h-16), 0.38, (80,40,10), 1)

        # Show (or freeze) the image
        if not paused or (paused and 'image' in locals()):
            cv2.imshow(window_name, image)
        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break
        elif key == ord(' '):  # Pause/unpause
            paused = not paused
        elif key == ord('s'):  # Screenshot
            img_count += 1
            cv2.imwrite(f"screenshot_{img_count}.png", image)
            print(f"Screenshot saved as screenshot_{img_count}.png")
        elif key == ord('c'):  # Color theme
            theme_idx = (theme_idx + 1) % len(color_themes)

cap.release()
cv2.destroyAllWindows()